In [4]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

real_path = r"C:\Users\pavli\for-2seconds\training\real"
fake_path = r"C:\Users\pavli\for-2seconds\training\fake"
real = []
fake = []

vreal_path = r"C:\Users\pavli\for-2seconds\mixed\real"
vfake_path = r"C:\Users\pavli\for-2seconds\mixed\fake"
vreal = []
vfake = []

treal_path = r"C:\Users\pavli\for-2seconds\testing\real"
tfake_path = r"C:\Users\pavli\for-2seconds\testing\fake"
treal = []
tfake = []


In [5]:
def get_spectrogram(folder_path):
    specs = []    
    for filename in os.listdir(folder_path):
        audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
        audio_parts = np.array_split(audio, 4)
        for part in audio_parts:
            spectrogram = abs(librosa.stft(y=part))
            spectrogram = librosa.amplitude_to_db(spectrogram, ref=np.max)
            specs.append(spectrogram)
    return specs

In [6]:
realS = get_spectrogram(real_path)
fakeS = get_spectrogram(fake_path)

vrealS = get_spectrogram(vreal_path)
vfakeS = get_spectrogram(vfake_path)

trealS = get_spectrogram(treal_path)
tfakeS = get_spectrogram(tfake_path)

In [25]:
len(realS)

27912

In [36]:
import numpy as np

def split_of_4(array, ratio=0.25):
    N = len(array)
    split_size = int(ratio * N) 
    split_size = split_size - (split_size % 4)  

    part1 = array[split_size:]  
    part2 = array[:split_size]  

    return part1, part2

X_trainR, X_ensembleR = split_of_4(realS)
X_trainF, X_ensembleF = split_of_4(fakeS)


print("Розмір першої частини:", len(X_trainR))  
print("Розмір другої частини:", len(X_ensembleR))  


Розмір першої частини: 20936
Розмір другої частини: 6976


In [55]:
from sklearn.model_selection import train_test_split

real_labels = np.zeros(len(X_trainR))
fake_labels = np.ones(len(X_trainF))

vreal_labels = np.zeros(len(vrealS))
vfake_labels = np.ones(len(vfakeS))

X_train = np.concatenate((X_trainR, X_trainF), axis=0)
y_train = np.concatenate((real_labels, fake_labels), axis=0)

X_val = np.concatenate((vrealS, vfakeS), axis=0)
y_val = np.concatenate((vreal_labels, vfake_labels), axis=0)

# Перемішуємо тренувальні дані
shuffle_index_train = np.random.permutation(len(X_train))
X_train_shuffled = X_train[shuffle_index_train]
y_train_shuffled = y_train[shuffle_index_train]

# Перемішуємо валідаційні дані
shuffle_index_val = np.random.permutation(len(X_val))
X_val_shuffled = X_val[shuffle_index_val]
y_val_shuffled = y_val[shuffle_index_val]


# Виведення розмірності тренувальних та тестових даних

print("Розмірність тренувальних даних:", X_train_shuffled.shape)
print("Розмірність тестових даних:", X_val_shuffled.shape)
print("Розмірність міток тренувальних даних:", y_train_shuffled.shape)
print("Розмірність міток тестових даних:", y_val_shuffled.shape)

X_train = X_train_shuffled[..., np.newaxis]
X_test = X_val_shuffled[..., np.newaxis]

Розмірність тренувальних даних: (41872, 1025, 16)
Розмірність тестових даних: (12912, 1025, 16)
Розмірність міток тренувальних даних: (41872,)
Розмірність міток тестових даних: (12912,)


In [57]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(X_train.shape[1:])),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train, y_train_shuffled, epochs=10, batch_size=32, validation_data=(X_test, y_val_shuffled))


Epoch 1/10
1309/1309 ━━━━━━━━━━━━━━━━━━━━ 35s 26ms/step - accuracy: 0.6265 - loss: 2.9581 - val_accuracy: 0.8052 - val_loss: 0.4320
Epoch 2/10
1309/1309 ━━━━━━━━━━━━━━━━━━━━ 35s 27ms/step - accuracy: 0.8075 - loss: 0.4195 - val_accuracy: 0.8595 - val_loss: 0.3124
Epoch 3/10
1309/1309 ━━━━━━━━━━━━━━━━━━━━ 33s 25ms/step - accuracy: 0.8866 - loss: 0.2675 - val_accuracy: 0.8990 - val_loss: 0.2316
Epoch 4/10
1309/1309 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - accuracy: 0.9169 - loss: 0.2001 - val_accuracy: 0.8704 - val_loss: 0.3004
Epoch 5/10
1309/1309 ━━━━━━━━━━━━━━━━━━━━ 33s 26ms/step - accuracy: 0.9282 - loss: 0.1820 - val_accuracy: 0.9057 - val_loss: 0.2220
Epoch 6/10
1309/1309 ━━━━━━━━━━━━━━━━━━━━ 32s 25ms/step - accuracy: 0.9374 - loss: 0.1588 - val_accuracy: 0.9200 - val_loss: 0.1963
Epoch 7/10
1309/1309 ━━━━━━━━━━━━━━━━━━━━ 35s 27ms/step - accuracy: 0.9477 - loss: 0.1401 - val_accuracy: 0.9074 - val_loss: 0.2510
Epoch 8/10
1309/1309 ━━━━━━━━━━━━━━━━━━━━ 34s 26ms/step - accuracy: 0.9567 -

In [59]:
X_ensembleRSpred = model.predict(np.array(X_ensembleR))
X_ensembleFSpred = model.predict(np.array(X_ensembleF))

vrealSpred = model.predict(np.array(vrealS))
vfakeSpred = model.predict(np.array(vfakeS))

trealSpred = model.predict(np.array(trealS))
tfakeSpred = model.predict(np.array(tfakeS))

218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
202/202 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [61]:
import pandas as pd
pd.DataFrame(X_ensembleRSpred).to_csv('X_ensembleRSpred.csv', index=False)
pd.DataFrame(X_ensembleFSpred).to_csv('X_ensembleFSpred.csv', index=False)
pd.DataFrame(vrealSpred).to_csv('vrealSpred.csv', index=False)
pd.DataFrame(vfakeSpred).to_csv('vfakeSpred.csv', index=False)
pd.DataFrame(trealSpred).to_csv('trealSpred.csv', index=False)
pd.DataFrame(tfakeSpred).to_csv('tfakeSpred.csv', index=False)

In [16]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

def reshape_to_blocks_of_4(array):
    return array.reshape(-1, 4)

train_predR = reshape_to_blocks_of_4(pd.read_csv('VpredR.csv').values)
train_predF = reshape_to_blocks_of_4(pd.read_csv('VpredF.csv').values)
train_LSTMpredR = reshape_to_blocks_of_4(pd.read_csv('LSTM_VpredR.csv').values)
train_LSTMpredF = reshape_to_blocks_of_4(pd.read_csv('LSTM_VpredF.csv').values)

test_predR = reshape_to_blocks_of_4(pd.read_csv('predR.csv').values)
test_predF = reshape_to_blocks_of_4(pd.read_csv('predF.csv').values)
test_LSTMpredR = reshape_to_blocks_of_4(pd.read_csv('LSTM_predR.csv').values)
test_LSTMpredF = reshape_to_blocks_of_4(pd.read_csv('LSTM_predF.csv').values)


In [17]:
X_train = np.vstack((np.hstack((train_predR, train_LSTMpredR)),  
                          np.hstack((train_predF, train_LSTMpredF))))  

X_test = np.vstack((np.hstack((test_predR, test_LSTMpredR)),  
                         np.hstack((test_predF, test_LSTMpredF))))  

y_train = np.vstack((np.ones((train_predR.shape[0], 1)),  
                          np.zeros((train_predF.shape[0], 1))))  

y_test = np.vstack((np.ones((test_predR.shape[0], 1)),  
                         np.zeros((test_predF.shape[0], 1))))  

In [18]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.30, random_state=42)

In [23]:
mlp_model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),

    Dense(64, activation='relu'),
    BatchNormalization(),
    
    Dense(1, activation='sigmoid')
])
optimizer = Adam(learning_rate=0.0005)
mlp_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

mlp_model.fit(X_train, y_train, 
              epochs=10, batch_size=512, 
              validation_data=(X_valid, y_),
              verbose=1)


mlp_accuracy = mlp_model.evaluate(X_valid, y_valid, verbose=0)[1]

print(f"Точність Покращеної MLP: {mlp_accuracy * 100:.2f}%")

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.7935 - loss: 0.5970 - val_accuracy: 0.6073 - val_loss: 0.6998
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8824 - loss: 0.3718 - val_accuracy: 0.7441 - val_loss: 0.6791
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9365 - loss: 0.2430 - val_accuracy: 0.8196 - val_loss: 0.6593
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9659 - loss: 0.1935 - val_accuracy: 0.8809 - val_loss: 0.6402
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9744 - loss: 0.1529 - val_accuracy: 0.9127 - val_loss: 0.6221
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9787 - loss: 0.1337 - val_accuracy: 0.9387 - val_loss: 0.6052
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9836 - loss: 0.1191 - val_accuracy: 0.9517 - val_loss: 0.5890
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9857 - loss: 0.1096 - val_accuracy: 0.9611 - val_loss: 0.5737
Epoch 9